# Model Architecture

In [1]:
from keras.layers import Input, Embedding, BatchNormalization, GRU, Dense, merge, Permute, RepeatVector, TimeDistributed
from keras.models import Model
from keras import optimizers

/home/richard/deep_learn_p3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
LATENT_DIM = 300
BODY_LENGTH = 70
NUM_ENCODER_TOKENS = 8000
NUM_DECODER_TOKENS = 4500

#### Encoder ####
encoder_inputs = Input(shape=(BODY_LENGTH,), name='Encoder-Input')

x = Embedding(NUM_ENCODER_TOKENS, LATENT_DIM, mask_zero=False, name='Body-Word-Embedding')(encoder_inputs)
x = BatchNormalization(name='Encoder-BatchNorm-1')(x)

_, state_h = GRU(LATENT_DIM, return_state=True, name='Encoder-Last-GRU')(x)
_, state_h_back = GRU(LATENT_DIM, return_state=True, go_backwards=True, name='Encoder-Last-Backward-GRU')(x)

encoder_model = Model(encoder_inputs, state_h, name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs)

#### Decoder ####
decoder_inputs = Input(shape=(None,), name='Decoder-Input')

x = Embedding(NUM_DECODER_TOKENS, LATENT_DIM, mask_zero=False, name='Title-Word-Embedding')(decoder_inputs)
x = BatchNormalization(name='Decoder-BatchNorm-1')(x)

decoder_gru = GRU(LATENT_DIM, return_state=True, return_sequences=True, name='Decoder-GRU')

decoder_gru_out, _ = decoder_gru(x, initial_state=seq2seq_encoder_out)

x = TimeDistributed(BatchNormalization(), name='Decoder-BatchNorm-2')(decoder_gru_out)

decoder_dense_out = Dense(NUM_DECODER_TOKENS, activation='softmax', name='Final-Output-Dense')(x)

#### Seq2Seq Model ####
seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_dense_out, name='Seq2Seq-Model')

seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=1e-3), loss='sparse_categorical_crossentropy')

In [2]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input

class Predictor():
    def __init__(self, seq2seqModel, tk_body, tk_title, maxlen_body, maxlen_title):
        self.encoder_model = self.get_encoder_model(seq2seqModel)
        self.decoder_model = self.get_decoder_model(seq2seqModel)
        self.tk_body = tk_body
        self.tk_title = tk_title
        self.maxlen_body = maxlen_body
        self.maxlen_title = maxlen_title
        
    def get_encoder_model(self, model):
        return model.get_layer('Encoder-Model')
    
    def get_decoder_model(self, model):
        latent_dim = model.get_layer('Title-Word-Embedding').output_shape[-1]
    
        decoder_inputs = model.get_layer('Decoder-Input').input
        dec_emb = model.get_layer('Title-Word-Embedding')(decoder_inputs)
        dec_bn = model.get_layer('Decoder-BatchNorm-1')(dec_emb)

        gru_inference_state_input = Input(shape=(latent_dim,), name='hidden_state_input')

        gru_out, gru_state_out = model.get_layer('Decoder-GRU')([dec_bn, gru_inference_state_input]) 

        dec_bn2 = model.get_layer('Decoder-BatchNorm-2')(gru_out)
#         dec_bn2 = model.layers[-2](gru_out)

        dense_out = model.get_layer('Final-Output-Dense')(dec_bn2)

        decoder_model = Model([decoder_inputs, gru_inference_state_input],
                              [dense_out, gru_state_out])
        return decoder_model
    
    def create_title(self, body_text, original_title_text=None):
        word2idx_title = self.tk_title.word_index
        idx2word_title = {v: k for k, v in word2idx_title.items()}

        raw_tokenized = self.tk_body.texts_to_sequences([body_text])
        raw_tokenized = pad_sequences(raw_tokenized, self.maxlen_body)
        body_encoding = self.encoder_model.predict(raw_tokenized)

        original_body_encoding = body_encoding
        state_value = np.array(word2idx_title["'start'"]).reshape(1, 1)        

        decoded_sentence = []
        stop_condition = False
        while not stop_condition:
            preds, st = self.decoder_model.predict([state_value, body_encoding])

            pred_idx = np.argmax(preds[:, :, 2:]) + 2

            pred_word_str = idx2word_title[pred_idx]

            if pred_word_str == "'end'" or len(decoded_sentence) >= self.maxlen_title:
                stop_condition = True
                break
            decoded_sentence.append(pred_word_str)

            # update the decoder for the next word
            body_encoding = st
            state_value = np.array(pred_idx).reshape(1, 1)

        return ' '.join(decoded_sentence)

In [61]:
p = Predictor(seq2seq_Model, tk_body, tk_title, BODY_LENGTH, 12)

for sample in SAMPLE_BODIES:
    print(p.create_title(sample))

writing delayed scroll day lazy lazy xp tile rather websites krav locked
editors lifecycle wizard zero tc portal tela unused detect angular2 reflection lookup
editors met learn multiline pool factory association wiki mapping corrupted gone posts
consume mac tokens 38 data data rendering rendering prefixes figure julia training


In [62]:
seq2seq_Model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Title-Word-Embedding (Embedding (None, None, 300)    1350000     Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, 70)           0                                            
__________________________________________________________________________________________________
Decoder-BatchNorm-1 (BatchNorma (None, None, 300)    1200        Title-Word-Embedding[0][0]       
__________________________________________________________________________________________________
Encoder-Mo

In [63]:
from keras.utils import plot_model
plot_model(seq2seq_Model, to_file='model.png')

# Train Model

In [64]:
SAMPLE_BODIES = ["""
hi, i'm trying to get the deep link working. 
i can send the activity, open the app and read dashcode and get booleanextra and all that. 
so activating the deep link works fine and for example when i call getaction, 
it returns android.intent.action.view which is correct. 
the main problem is that getdatastring and getscheme always return null. 
i'm out of test ideas. do you think its a bug? i have attached the manifest file for your reference. 
and i'm using gvrintent.getdata that always returns null. 
islaunchedfromvr and getintenthashcode are working fine. 
and this is the command line i used to test as an example: 
./adb shell am start -w -a android.intent.action.view -d shapevisual://com.shapevisual.app?wl=gfs com.shapevisual.app androidmanifest.xml.txt https://github.com/googlevr/gvr-unity-sdk/files/864522/androidmanifest.xml.txt
""",
"""
issue overview add a new property to disable detection of image stream files those ended with -is.yml 
from target directory. expected behaviour by default cube should not process image stream files 
if user does not set it. current behaviour cube always try to execute -is.yml files 
which can cause some problems in most of cases, for example if you are using kuberentes 
instead of openshift or if you use together fabric8 maven plugin with cube. 
""",
"""
i'm using highstockcharts and recently upgraded to v6.0.3. since then, the tooltips won't be shown anymore 
as soon as the tooltip is higher than the actual chart. see the minimum example which i've provided. 
expected behaviour the tooltip should be shown. actual behaviour the tooltip is not shown if the tooltip 
the height is larger than the actual chart. live demo with steps to reproduce http://jsfiddle.net/n1h3q3sr/ 
uncomment the part teststring += <br/> not working anymore to make the tooltip visible. 
affected browser s chrome / firefox and most probably ie too
""",
"""
i created a form 'contact' and it seems to work but the form labels do not appear on the form 
so it is a bit useless. please get the labels to appear and merge and release with other improvements asap
"""]

In [3]:
from IPython.display import Audio, display
from Helpers import load_tokenizer
from keras.callbacks import Callback

tk_body = load_tokenizer('tk_body.dpkl')
tk_title = load_tokenizer('tk_title.dpkl')


class LossHistory(Callback):
    def on_epoch_end(self, epoch, logs):
        display(Audio(filename='notification.mp3', autoplay=True))
        print('\n')
        print('epoch: {}'.format(epoch))
        print('val_loss: {}'.format(logs.get('val_loss')))
        
        predictor = Predictor(seq2seq_Model, tk_body, tk_title, BODY_LENGTH, 12)
        
        for body in SAMPLE_BODIES:
            title = predictor.create_title(body)
            print('Generated title: {}'.format(title))

Size of vocabulary for tk_body.dpkl: 8000
Size of vocabulary for tk_title.dpkl: 4500


In [19]:
from Helpers import load_encoder_inputs, load_decoder_inputs

encoder_input_data, _ = load_encoder_inputs('train_body_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs('train_title_vecs.npy')

Shape of encoder input: (1800000, 70)
Shape of decoder input: (1800000, 11)
Shape of decoder target: (1800000, 11)


In [66]:
import numpy as np
from keras.callbacks import ModelCheckpoint

batch_size = 1200
epochs = 8
history = LossHistory()
checkpointer = ModelCheckpoint(filepath='/tmp/models_bi1_tdBatchNorm2.{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1)

seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
                   batch_size=batch_size,
                   epochs=epochs,
                   validation_split=0.12, 
                   callbacks=[history, checkpointer])

Train on 1584000 samples, validate on 216000 samples
Epoch 1/8
1582800/1584000 [============================>.] - ETA: 0s - loss: 2.6045



epoch: 0
val_loss: 2.3239502323998344
Generated title: android sdk fails to work
Generated title: disable detection of images
Generated title: tooltip tooltip is not visible
Generated title: form labels are not working
Epoch 00001: saving model to /tmp/models_bi1_tdBatchNorm2.01-2.32.hdf5


/home/richard/deep_learn_p3/lib/python3.5/site-packages/keras/engine/topology.py:2344: UserWarning: Layer Decoder-GRU was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder-Model_3/Encoder-Last-GRU/while/Exit_2:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


1584000/1584000 [==============================] - 304s 192us/step - loss: 2.6042 - val_loss: 2.3240
Epoch 2/8
1582800/1584000 [============================>.] - ETA: 0s - loss: 2.2403



epoch: 1
val_loss: 2.2535875346925525
Generated title: android app not working
Generated title: detection of images in yml
Generated title: tooltip not working
Generated title: form labels are not
Epoch 00002: saving model to /tmp/models_bi1_tdBatchNorm2.02-2.25.hdf5
1584000/1584000 [==============================] - 307s 194us/step - loss: 2.2403 - val_loss: 2.2536
Epoch 3/8
1582800/1584000 [============================>.] - ETA: 0s - loss: 2.1586



epoch: 2
val_loss: 2.2260388771692914
Generated title: android app crashes when using
Generated title: detection of images
Generated title: tooltip not working on the chart
Generated title: form labels are not appearing
Epoch 00003: saving model to /tmp/models_bi1_tdBatchNorm2.03-2.23.hdf5
1584000/1584000 [==============================] - 309s 195us/step - loss: 2.1585 - val_loss: 2.2260
Epoch 4/8
1582800/1584000 [============================>.] - ETA: 0s - loss: 2.1110



epoch: 3
val_loss: 2.2150964736938477
Generated title: android app crashes when using
Generated title: detection of images by default
Generated title: tooltip not working
Generated title: form labels are not shown
Epoch 00004: saving model to /tmp/models_bi1_tdBatchNorm2.04-2.22.hdf5
1584000/1584000 [==============================] - 311s 196us/step - loss: 2.1110 - val_loss: 2.2151
Epoch 5/8
1582800/1584000 [============================>.] - ETA: 0s - loss: 2.0771



epoch: 4
val_loss: 2.2134512742360433
Generated title: null reference in
Generated title: image detection should be ignored by default
Generated title: tooltip not working
Generated title: form labels are not
Epoch 00005: saving model to /tmp/models_bi1_tdBatchNorm2.05-2.21.hdf5
1584000/1584000 [==============================] - 313s 198us/step - loss: 2.0771 - val_loss: 2.2135
Epoch 6/8
  37200/1584000 [..............................] - ETA: 4:56 - loss: 2.0011

KeyboardInterrupt: 

# See Results

In [8]:
from keras.models import load_model

p1 = Predictor(load_model('/tmp/models_bi1_tdBatchNorm2.01-2.32.hdf5'), 
               tk_body, tk_title, BODY_LENGTH, 12)
p2 = Predictor(load_model('/tmp/models_bi1_tdBatchNorm2.02-2.25.hdf5'), 
               tk_body, tk_title, BODY_LENGTH, 12)
p3 = Predictor(load_model('/tmp/models_bi1_tdBatchNorm2.03-2.23.hdf5'), 
               tk_body, tk_title, BODY_LENGTH, 12)
p4 = Predictor(load_model('/tmp/models_bi1_tdBatchNorm2.04-2.22.hdf5'), 
               tk_body, tk_title, BODY_LENGTH, 12)
p5 = Predictor(load_model('/tmp/models_bi1_tdBatchNorm2.05-2.21.hdf5'), 
               tk_body, tk_title, BODY_LENGTH, 12)

In [16]:
SAMPLE_BODIES = [
    """
I am developing a neural network in order to classify timeseries data. I know for timeseries LSTM would be right approach but I am using https://github.com/cerndb/dist-keras where before passing it to a trainer dataset has to be spark dataframe format.

I am following this example http://vict0rsch.github.io/tutorials/keras/recurrent/ and task here is to port this example to dist-keras. Timestep is 50 which means model would take 0-49 and predict 50 and so on. As you can see in the example that data is being pre-processed using numpy before being fed to keras. Since dist-keras requires data to be in spark dataframe, I have to take a different approach which is as follows:

I have straightaway created the DF:

X_train = train[:, :] y_train = train[:, -1] raw_dataset_train = sc.createDataFrame(X_train.tolist())

Above code will create a DF having 50 columns(timestep is 50) from 0 to _50.

Remove the _50 column which is the label in our case and then applying the vector assembler to all features:

features = raw_dataset_train.columns features.remove('_50') vector_assembler = VectorAssembler(inputCols=features, outputCol="features") dataset_train = vector_assembler.transform(raw_dataset_train)

Now, each row of DF contains 2 columns. First column contains the features and second contains the label(_50 column which I want to train on and later predict on). As I see it, it become a classification problem. My issues are below:

If my approach is right, then how would I defines output label for my data as their are no finite number here for output column. It could be same number as number of rows in DF.

Do I still need LSTM layers in my model? I am asking this because I have processed the data in non-lstm kind of way.(At least that is what I think. I might be wrong.)

Please advice and let me know if you need more clarification or information on this.
    """,
    """
I'm trying to use Intel NLP Architect and its example has this line that makes it not working:

>>> from keras.engine import saving
Using TensorFlow backend.
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ImportError: cannot import name 'saving'
Anyone knows how to fix it? Thank you!
Also this is a link to the whole framework: http://nlp_architect.nervanasys.com/ner_crf.html But the line that makes it not working is just that one line.
    """,
    """
Can multiple barcodes be scanned at once?
For example, scanning a sheet of paper with multiple barcodes printed on it.
If not possible, any recommendations on how to do it would be greatly appreciated.
    """,
    """
I call getSupportedRatiosAsync for Android setup, which (I guess) result in crash as follows.

react-native-camera/android/src/main/java/org/reactnative/camera/RNCameraView.java

Line 191 in 60b9ef1

 preview.layout(paddingX, paddingY, correctWidth + paddingX, correctHeight + paddingY); 
I get width / height ratio list and set camera aspect ratio to closest one. Some phone generates this error which I found Crashlytics. I guess this line needs some protective code.
    """,
    """
Hello everyone!
I wanted to open this "thread" to get some feedback on a new implementation of react-native-modal I'm building.

As you may already know, the current version of react-native-modal is based on react-native's original modal, which, unfortunately, has many different issues that are difficult to solve since it would involve native code.

To completely solve (almost all of) these issues I'm building a 100% Javascript version of react-native-modal that doesn't use the react-native's original modal.
The main issue of using a total JS solution is that you should place it on the root of you app to make it work (because otherwise the modal won't be able to overlay the entire screen), but I'm testing a portal-like solution that should overcome this block (and it seems to be working!).

Are you interested in it?
Do you think it would solve any of your issues with the current modal?
Do you think it would be useful releasing this WIP in a separate beta branch?

Any feedback is welcome!
    """,
    """
As the title suggest, in my opinion it is better to have react-native-animatable as a peer dependency instead.

I think it is reasonable enough as react-native-animatable is a frequently used library in my experience. The motivation is of course to eliminate unnecessary duplications and move the control over to the user of this library. react-native-animatable changes quite infrequently (judging by the releases on github), so it is very likely that both users' local version and react-native-modal's version will semantically match and it won't cause any pains.

Let me know what you think. Cheers!
    """
]

for body in SAMPLE_BODIES:
    print(p1.create_title(body))
    print(p2.create_title(body))
    print(p3.create_title(body))
    print(p4.create_title(body))
    print(p5.create_title(body))
    print('\n')

lstm data model
question how to use lstm
question about lstm
lstm
how to use lstm


cannot import
import error
cannot import
error with
import error


multiple sheets
multiple sheets with multiple
multiple on same sheet
multiple sheets with multiple
multiple on same sheet


android crash
camera aspect ratio
camera aspect ratio
camera aspect ratio
android camera preview


modal for
modal for
modal for
modal dialog
modal overlay


react native
react native version
react native
react native 0 40
react native


